In [ ]:
import cv2
import numpy as np
import os
import math
from src.our_chessboard_detection import *
from matplotlib import pyplot as plt

dataDir = 'images/' # Change this, according to your images' directory path

In [ ]:
# Open image
image = cv2.imread(os.path.join(dataDir, 'G000_IMG062.jpg')) # Change this, according to your image's path
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# cv2.imshow("Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
plt.imshow(rgb)

### applying different filters

In [ ]:
filtered_images = apply_filters(image, True)

### Detecting chessboard

In [ ]:
chess_contour = get_contours(filtered_images, show=True,  kernel_size=(25,25) ,  kernel_usage=True, iterations=4)

### rotate and warp

In [ ]:
warped_image, M = rotate_and_crop(filtered_images, chess_contour[0][1], show=True)

In [ ]:

# Find the best rotation angle
rotated_image, best_angle = align_board(warped_image, radius=12, angle_step=90)

# Show the rotated image and the best angle
plt.imshow(rotated_image, cmap='gray')
plt.axis('off')
plt.show()


In [ ]:
print(best_angle)

In [ ]:
grid_image = chesboard_grids(rotated_image, show = True)

In [ ]:
def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def adjust_contrast(image, alpha=1.5, beta=0):
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted

def normalize_white_pieces(square_rgb):
    hsv = cv2.cvtColor(square_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    mask = (
        (h >= 15) & (h <= 45) &
        (s >= 15) & (s <= 200) &
        (v >= 100) & (v <= 240)
    )
    hsv[mask] = (30, 180, 200)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

def display_chessboard_squares(warped, gamma=1.5):
    img_grid = warped.copy()
    squares = []

    for row in range(8):
        for col in range(8):
            x1 = margin + col * square_size
            y1 = margin + row * square_size
            x2 = x1 + square_size
            y2 = y1 + square_size

            square = warped[y1:y2, x1:x2]
            square_rgb = cv2.cvtColor(square, cv2.COLOR_BGR2RGB)
            image_light = adjust_gamma(square_rgb, gamma)
            image_norm = normalize_white_pieces(image_light)
            image_contrast = adjust_contrast(image_norm, alpha=1.3, beta=0)
            image_filtered = cv2.medianBlur(image_contrast, 5)
            squares.append(image_filtered)

    fig, axes = plt.subplots(8, 8, figsize=(10, 10))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(squares[i])
        ax.axis('off')
        ax.set_title(f"{i}", fontsize=10)

    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    plt.show()

    return squares

In [ ]:
squares = display_chessboard_squares(rotated_image)

In [ ]:
def check_piece_at_center(square, black_piece=(45, 45, 45), white_piece=(255, 252, 94)):
    
    # Center of the square
    center_x = square.shape[1] // 2
    center_y = square.shape[0] // 2

    # Crop a small region around the center 3x3
    crop_size = 3 
    crop_x1 = max(center_x - crop_size // 2, 0)
    crop_y1 = max(center_y - crop_size // 2, 0)
    crop_x2 = min(center_x + crop_size // 2, square.shape[1])
    crop_y2 = min(center_y + crop_size // 2, square.shape[0])

    # Crop the center region
    cropped_region = square[crop_y1:crop_y2, crop_x1:crop_x2]

    # Calculate the average color of the center
    avg_color = np.mean(cropped_region, axis=(0, 1))

    # Compare the average color with the predefined values
    if np.allclose(avg_color, white_piece, atol=50):  # allow a tolerance for slight color variations
        return "WHITE"
    elif np.allclose(avg_color, black_piece, atol=50):
        return "BLACK"
    else:
        return "EMPTY"

In [ ]:
def process_chessboard(squares):
    # Initialize matrix for 8x8 board and counters for white and black pieces
    board_matrix = np.zeros((8, 8))
    white_count = 0
    black_count = 0

    for i in range(8):
        for j in range(8):
            square = squares[i * 8 + j]

            # Check the piece at the center of the square
            piece_type = check_piece_at_center(square)

            # If the piece is a WHITE or BLACK
            if piece_type == "WHITE" or piece_type == "BLACK":
                board_matrix[i, j] = 1
                # Count white and black pieces
                if piece_type == "WHITE":
                    white_count += 1
                elif piece_type == "BLACK":
                    black_count += 1

    # Print the board matrix
    print("Board Matrix (8x8):")
    print(board_matrix)

    # Print the counts of cream and black pieces
    print(f"White pieces: {white_count}")
    print(f"Black pieces: {black_count}")

    return board_matrix

In [ ]:
board_matrix = process_chessboard(squares)

In [ ]:
def reverse_piece_coordinates(piece_coords, rotation_angle, perspective_matrix, rotated_image_shape):
    """
    Reverses perspective and rotation transforms to map piece bounding boxes
    back to coordinates in the original image.

    Args:
        piece_coords: list of dicts with keys 'xmin', 'xmax', 'ymin', 'ymax'
        rotation_angle: angle used to rotate the warped image
        perspective_matrix: M used for perspective warp (cv2.getPerspectiveTransform)
        rotated_image_shape: shape of the rotated (aligned) image

    Returns:
        A list of dictionaries with original image coordinates using the same bounding box keys.
    """
    h_rot, w_rot = rotated_image_shape[:2]
    center = (w_rot // 2, h_rot // 2)

    # Create inverse rotation matrix
    R = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    R_3x3 = np.vstack([R, [0, 0, 1]])
    R_inv = np.linalg.inv(R_3x3)

    # Collect all corner points from bounding boxes
    all_points = []
    box_indices = []

    for idx, box in enumerate(piece_coords):
        corners = [
            [box['xmin'], box['ymin']],
            [box['xmax'], box['ymin']],
            [box['xmax'], box['ymax']],
            [box['xmin'], box['ymax']]
        ]
        all_points.extend(corners)
        box_indices.extend([idx] * 4)

    # Apply inverse rotation
    points_np = np.array(all_points, dtype=np.float32)
    points_hom = np.hstack([points_np, np.ones((len(points_np), 1))])
    coords_in_warped = (R_inv @ points_hom.T).T[:, :2]

    # Apply inverse perspective
    M_inv = np.linalg.inv(perspective_matrix)
    coords_in_warped = np.array(coords_in_warped, dtype=np.float32).reshape(-1, 1, 2)
    coords_in_original = cv2.perspectiveTransform(coords_in_warped, M_inv).reshape(-1, 2)

    # Group back into box dictionaries
    original_boxes = [{} for _ in range(len(piece_coords))]
    for i in range(len(piece_coords)):
        box_points = coords_in_original[i * 4: (i + 1) * 4]
        x_vals = box_points[:, 0]
        y_vals = box_points[:, 1]
        original_boxes[i] = {
            "xmin": int(np.min(x_vals)),
            "ymin": int(np.min(y_vals)),
            "xmax": int(np.max(x_vals)),
            "ymax": int(np.max(y_vals))
        }

    return original_boxes

In [ ]:
# Build list of centers of squares with pieces
piece_coords = []
for i in range(8):
    for j in range(8):
        if board_matrix[i, j] == 1:
            x1 = margin + j * square_size
            y1 = margin + i * square_size
            x2 = x1 + square_size
            y2 = y1 + square_size

            piece_coords.append({
                "xmin": int(x1),
                "ymin": int(y1),
                "xmax": int(x2),
                "ymax": int(y2)
            })


# Reverse transforms
original_coords = reverse_piece_coordinates(
    piece_coords,
    rotation_angle=best_angle,  # from align_board
    perspective_matrix=M,       # from rotate_and_crop
    rotated_image_shape=rotated_image.shape
)

print(original_coords)